# Example: Hyperparameter optimization with Optuna

This notebook introduces 3 examples für hyperparameter optimization based on different optimization objectives.
1. Optimizing the Autoencoder reconstruction using the MSE
2. Optimizing the FaultDetector classification performance using the Fbeta score
3. Optimizing the FaultDetector classification performance using the CARE-score
The optimization is done using the [CARE to Compare dataset](https://doi.org/10.5281/zenodo.14958989)

For this example you need to install Optuna, which is not contained in the standard requirements of the framework
Optuna [docs](https://optuna.readthedocs.io/en/stable/index.html) and [tutorials](https://optuna.readthedocs.io/en/stable/tutorial/index.html)

-> Install additional requirements for this example using 'pip notebooks/example_requirements.txt'

In [ ]:
from typing import List

import optuna as op
import pandas as pd
import numpy as np
from sklearn.metrics import fbeta_score

from energy_fault_detector.fault_detector import FaultDetector
from energy_fault_detector.config import Config
from energy_fault_detector.evaluation import CAREScore, Care2CompareDataset

In [ ]:
data_path = './Care_To_Compare'

In [ ]:
def update_config(config: Config, feature_descriptions: pd.DataFrame) -> None:
    """Update config based on provided feature descriptions."""

    def get_columns(feature_description_selection: pd.DataFrame) -> List[str]:
        col_suffix = {
            'average': 'avg',
            'minimum': 'min',
            'maximum': 'max',
            'std_dev': 'std'
        }
        columns = []
        for _, row in feature_description_selection.iterrows():
            if row.statistics_type == 'average':
                # in this case the column can be either sensor_i or sensor_i_avg, so we add both
                columns.append(row.sensor_name)
            for stat in row.statistics_type.split(','):
                columns.append(f'{row.sensor_name}_{col_suffix[stat]}')
        return columns

    angles = feature_descriptions.loc[feature_descriptions['is_angle']]
    to_exclude = feature_descriptions.loc[feature_descriptions['is_counter']]

    angle_columns = get_columns(angles)
    to_exclude_columns = get_columns(to_exclude)
    
    config['train']['data_preprocessor']['params']['angles'] = (
        config['train']['data_preprocessor']['params'].get('angles', []) + angle_columns
    )
    config['train']['data_preprocessor']['params']['features_to_exclude'] = (
        config['train']['data_preprocessor']['params'].get('features_to_exclude', []) + to_exclude_columns
    )
    
    config.update_config(config.config_dict)


In [ ]:
c2c = Care2CompareDataset(data_path)

### Optimize autoencoder reconstruction

In [ ]:
model_config = Config('c2c_configs/windfarm_C.yaml')  # starting point

# our test set
c2c = Care2CompareDataset(data_path)
event_id = 47
train_data, normal_index, _, _ = c2c.get_formatted_event_dataset(event_id=event_id, index_column='time_stamp')

# speed up for testing
N = 10000
normal_index = normal_index.iloc[:N]
train_data = train_data.iloc[:N]

# Create an objective - what should be optimized? --> MSE of the reconstruction error
# NOTE:
# you can increase the speed of this part slightly if you do not need to fit the datapreprocessor.
# in that case, you would fit and apply the data preprocessor outside of this function and only fit the autoencoder inside the objective.
def reconstruction_mse(trial: op.Trial) -> float:
    """Samples new hyperparameters. fits a new model and returns the reconstruction error (MSE) of the validation data.

    Args:
        trial: optuna Trial object

    Returns:
        MSE of the reconstruction.
    """

    autoencoder_params = model_config.config_dict['train']['autoencoder']['params']

    # sample new parameters
    autoencoder_params['batch_size'] = int(trial.suggest_categorical(
        name='batch_size', choices=[32, 64, 128]
    ))
    autoencoder_params['learning_rate'] = trial.suggest_float(
        name='learning_rate', low=1e-5, high=0.01, log=True
    )
    autoencoder_params['decay_rate'] = trial.suggest_float(
        name='decay_rate', low=0.8, high=0.99
    )

    # architecture
    autoencoder_params['layers'][0] = trial.suggest_int(
        name='layers_0', low=100, high=400
    )
    autoencoder_params['layers'][1] = trial.suggest_int(
        name='layers_1', low=50, high=100
    )
    autoencoder_params['code_size'] = trial.suggest_int(
        name='code_size', low=10, high=30
    )

    # update the configuration
    model_config.update_config(model_config.config_dict)

    # create a new model using our new configuration and train the model
    model = FaultDetector(model_config)
    # For autoencoder optimization, we do not need to fit a threshold
    training_result = model.fit(train_data, normal_index=normal_index, fit_autoencoder_only=True, save_model=False)

    # Calculate the MSE of the reconstruction errors of the validation data - this is minimized
    deviations = training_result.val_recon_error
    score = np.mean((np.square(deviations)))

    return score

In [ ]:
study = op.create_study(sampler=op.samplers.TPESampler(),
                        study_name='autoencoder_optimization',
                        direction='minimize')

# if we want to ensure that the first trial is done with the hyperparameters of the configuration, we need to enqueue a trial:
autoencoder_params = model_config.config_dict['train']['autoencoder']['params']
study.enqueue_trial(params={
    'batch_size': autoencoder_params['batch_size'],
    'learning_rate': autoencoder_params['learning_rate'],
    'decay_rate': autoencoder_params['decay_rate'],
    'layers_0': autoencoder_params['layers'][0],
    'layers_1': autoencoder_params['layers'][1],
    'code_size': autoencoder_params['code_size'],
})

In [ ]:
study.optimize(reconstruction_mse, n_trials=5)

In [ ]:
study.best_params

In [ ]:
# analyze results
study.trials_dataframe()

# Optimize fault detection model

In [ ]:
c2c = Care2CompareDataset(data_path)

event_id = 47
event_info = c2c.event_info_all[c2c.event_info_all['event_id'] == event_id].iloc[0]

train_data, normal_index, test_data, test_normal_index = c2c.get_formatted_event_dataset(event_id=event_id, index_column='time_stamp')

ground_truth = CAREScore.create_ground_truth(
    event_label=event_info['event_label'],
    event_start=event_info['event_start'],
    event_end=event_info['event_end'],
    normal_index=test_normal_index
)

In [ ]:
model_config = Config('c2c_configs/windfarm_C.yaml')  # starting point

# speed up for testing
N = 10000
normal_index = normal_index.iloc[:N]
train_data = train_data.iloc[:N]


def f_score(trial: op.Trial) -> float:
    """Returns the F-score of the model (only useful for datasets with anomalies).

    Args:
        trial: optuna Trial object

    Returns:
        Score of the FaultDetector model 
    """

    dataprep_params = model_config.config_dict['train']['data_preprocessor']['params']
    autoencoder_params = model_config.config_dict['train']['autoencoder']['params']

    dataprep_params['scale'] = trial.suggest_categorical(
        name='scale', choices=['minmax', 'standardize']
    )

    autoencoder_params['batch_size'] = int(trial.suggest_categorical(
        name='batch_size', choices=[32, 64, 128]
    ))
    autoencoder_params['learning_rate'] = trial.suggest_float(
        name='learning_rate', low=1e-5, high=0.01, log=True
    )
    autoencoder_params['decay_rate'] = trial.suggest_float(
        name='decay_rate', low=0.8, high=0.99
    )

    # architecture
    autoencoder_params['layers'][0] = trial.suggest_int(
        name='layers_0', low=100, high=400
    )
    autoencoder_params['layers'][1] = trial.suggest_int(
        name='layers_1', low=50, high=100
    )
    autoencoder_params['code_size'] = trial.suggest_int(
        name='code_size', low=10, high=30
    )

    # update the configuration
    model_config.update_config(model_config.config_dict)

    # create a new model using our new configuration and train the model
    model = FaultDetector(model_config)
    _ = model.fit(train_data, normal_index=normal_index, save_models=False)
    predictions = model.predict(test_data)

    score = fbeta_score(
        y_true=ground_truth.sort_index(),
        y_pred=predictions.predicted_anomalies['anomaly'].sort_index(),
        beta=0.5
    )

    return score

In [ ]:
study = op.create_study(sampler=op.samplers.TPESampler(),
                        study_name='ad_optimization',
                        direction='maximize')

# if we want to ensure that the first trial is done with the hyperparameters of the configuration, we need to enqueue a trial:
autoencoder_params = model_config.config_dict['train']['autoencoder']['params']
study.enqueue_trial(params={
    'batch_size': autoencoder_params['batch_size'],
    'learning_rate': autoencoder_params['learning_rate'],
    'decay_rate': autoencoder_params['decay_rate'],
    'layers_0': autoencoder_params['layers'][0],
    'layers_1': autoencoder_params['layers'][1],
    'code_size': autoencoder_params['code_size'],
})

study.optimize(f_score, n_trials=5)

In [ ]:
study.trials_dataframe()

### Optimize CARE score
Optimize the CARE Score. Note that this is extremely slow, as we train a model for each subdataset.

In [ ]:
wind_farm = 'B'
model_config = Config('c2c_configs/windfarm_B.yaml')

# speed up for testing
N = 100

def care_objective(trial: op.Trial) -> float:
    """Returns the F-score of the model (only useful for datasets with anomalies).

    Args:
        trial: optuna Trial object

    Returns:
        Score of the FaultDetector model.
    """

    autoencoder_params = model_config.config_dict['train']['autoencoder']['params']
    threshold_params = model_config.config_dict['train']['threshold_selector']['params']

    autoencoder_params['batch_size'] = int(trial.suggest_categorical(
        name='batch_size', choices=[32, 64, 128]
    ))
    autoencoder_params['learning_rate'] = trial.suggest_float(
        name='learning_rate', low=1e-5, high=0.01, log=True
    )

    # architecture
    autoencoder_params['layers'][0] = trial.suggest_int(
        name='layers_0', low=20, high=100
    )
    autoencoder_params['code_size'] = trial.suggest_int(
        name='code_size', low=5, high=20
    )

    # threshold
    threshold_params['gamma'] = trial.suggest_float(name='gamma', low=0.05, high=0.3)
    threshold_params['nn_size'] = trial.suggest_int(name='nn_size', low=20, high=50)

    # update the configuration with the new hyperparameters
    model_config.update_config(model_config.config_dict)

    care_score = CAREScore(coverage_beta=0.5, eventwise_f_score_beta=0.5, anomaly_detection_method='criticality')
    i = 1
    for x_train, y_train, x_test, y_test, event_id in c2c.iter_formatted_datasets(wind_farm=wind_farm, index_column='time_stamp'):
        print(f"event {i}/{len(c2c.event_info_all[c2c.event_info_all['wind_farm'] == wind_farm])}")
        if N is not None:
            x_train = x_train.iloc[:N]
            x_test = x_test.iloc[:N]
            y_train = y_train.iloc[:N]
            y_test = y_test.iloc[:N]
        
        # create a new model using our new configuration and train the model
        model = FaultDetector(model_config)
        _ = model.fit(x_train, normal_index=y_train, save_models=False)
        prediction = model.predict(x_test)
        event_info = c2c.event_info_all[c2c.event_info_all['event_id'] == event_id].iloc[0]
        care_score.evaluate_event(
            event_id=event_id,
            event_start=event_info['event_start'],
            event_end=event_info['event_end'],
            event_label=event_info['event_label'],
            normal_index=y_test,
            predicted_anomalies=prediction.predicted_anomalies['anomaly'],
            ignore_normal_index=False
        )
        i += 1

    score = care_score.get_final_score()

    return score

In [ ]:
study = op.create_study(sampler=op.samplers.TPESampler(),
                        study_name='care_optimization',
                        direction='maximize')

# Ensure that the first trial is done with the hyperparameters of the provided configuration
autoencoder_params = model_config.config_dict['train']['autoencoder']['params']
threshold_params = model_config.config_dict['train']['threshold_selector']['params']
study.enqueue_trial(params={
    'batch_size': autoencoder_params['batch_size'],
    'learning_rate': autoencoder_params['learning_rate'],
    'layers_0': autoencoder_params['layers'][0],
    'code_size': autoencoder_params['code_size'],
    'gamma': threshold_params['gamma'],
    'nn_size': threshold_params['nn_size'],
})

# since we loop through many datasets, train many models, we run the garbage collector after each trial
study.optimize(care_objective, n_trials=5, gc_after_trial=True)

In [ ]:
study.trials_dataframe()